In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys, MufexTestKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.tv_indicators import *
from quantfreedom.plotting.plotting_base import *
from quantfreedom.exchanges.exchange import Exchange
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
from my_stuff import BybitTestKeys
from quantfreedom.exchanges.bybit_exchange.bybit import Bybit
from plotly.subplots import make_subplots


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol="BTCUSDT",
    timeframe="1h",
    # candles_to_dl=3000,
    since_date_ms=1626768000000,
    until_date_ms=1634745600000,
)
timestamps = candles[:, CandleBodyType.Timestamp]
open = candles[:, CandleBodyType.Open]
high = candles[:, CandleBodyType.High]
low = candles[:, CandleBodyType.Low]
close = candles[:, CandleBodyType.Close]
volume = candles[:, CandleBodyType.Volume]

In [3]:
candles_to_df(candles)

,timestamp,open,high,low,close,volume
datetime,,,,,,
2021-07-20 08:00:00,1626768000000,29850.80,29877.37,29632.59,29632.59,10.693783
2021-07-20 09:00:00,1626771600000,29683.30,29797.07,29312.84,29453.24,18.397659
2021-07-20 10:00:00,1626775200000,29394.79,29697.53,29309.24,29675.77,12.609095
2021-07-20 11:00:00,1626778800000,29697.59,29787.75,29584.11,29701.90,3.852527
2021-07-20 12:00:00,1626782400000,29701.91,29793.72,29623.27,29693.78,4.089045
...,...,...,...,...,...,...
2021-10-20 11:00:00,1634727600000,63817.22,63924.46,63715.84,63790.05,3.989312
2021-10-20 12:00:00,1634731200000,63791.97,66787.05,63763.38,64382.64,59.154814
2021-10-20 13:00:00,1634734800000,64388.65,66059.71,64165.39,66031.42,55.343934


In [4]:
volitility_lb = 30
bo_lb = 20

max_high = np.full_like(close, np.nan)

current_vol = np.std(close[0 : volitility_lb])

In [5]:
vol_lb_m_1 = volitility_lb - 1

for i in range(volitility_lb, close.size):
    prev_vol = current_vol
    current_vol = np.std(close[i - vol_lb_m_1 : i + 1])

    delta_vol = (current_vol - prev_vol) / current_vol

    real_bo_lb = bo_lb * (1 + delta_vol)
    max_high[i] = high[i - int(real_bo_lb) : i].max()
    
entry_prices = np.where(close >= max_high, close, np.nan)

In [6]:
datetimes = pd.to_datetime(candles[:, CandleBodyType.Timestamp], unit="ms")
scatter_dt = pd.to_datetime(candles[np.where(~np.isnan(entry_prices)), CandleBodyType.Timestamp], unit="ms")
fig = go.Figure(
    data=[
        go.Candlestick(
            x=datetimes,
            open=candles[:, 1],
            high=candles[:, 2],
            low=candles[:, 3],
            close=candles[:, 4],
            name="Candles",
        ),
        go.Scatter(
            x=pd.to_datetime(candles[np.where(~np.isnan(entry_prices))[0], CandleBodyType.Timestamp], unit="ms"),
            y=entry_prices[~np.isnan(entry_prices)],
            name="entries",
            mode="markers",
            marker=dict(color="yellow"),
        ),
    ]
)
fig.update_layout(height=800, xaxis_rangeslider_visible=False)
fig.show()